In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# Chapitre 6 — Initiation à l’algorithmique

Ce notebook introduit les notions d’**algorithme**, de **complexité** (notation de Landau / notation O)
et présente quelques **algorithmes de tri** (sélection, insertion, rapide).

Objectifs :

- comprendre ce qu’est un algorithme et les 5 conditions de Knuth ;
- comprendre la différence entre temps d’exécution, mémoire, etc. ;
- manipuler la notation O (`O`, `Θ`, `Ω`, ...) ;
- implémenter et analyser quelques algorithmes de tri.

Une section d’exercices supplémentaires à la fin pourra être corrigée par un **grader externe**.

## 6.1 Algorithmes — définition et conditions

En général, un **algorithme** est :

> une suite finie et non ambiguë d’opérations permettant de résoudre un problème ou d’obtenir un résultat.

En programmation, un algorithme est correct si, pour toutes les entrées possibles :

- il **se termine** ;
- il renvoie le **bon résultat**, ou un message d’erreur si les entrées ne conviennent pas.

Selon Donald Knuth, un algorithme doit satisfaire 5 conditions :

1. **Finitude** : il doit toujours se terminer après un nombre **fini** d’étapes ;
2. **Définition précise** : chaque étape doit être définie sans ambiguïté ;
3. **Entrées** : les données en entrée (arguments) doivent être clairement spécifiées ;
4. **Sorties** : il doit produire au moins une sortie, qui dépend des entrées si elles existent ;
5. **Efficacité** : chaque opération doit être suffisamment simple pour être réalisée à la main en temps fini.

Exemple : l’algorithme d’Euclide pour le **PGCD** satisfait ces 5 conditions.

### Exemple : algorithme d’Euclide (PGCD)

Pour deux entiers naturels non nuls `a` et `b` :

- si `b` divise `a`, alors `pgcd(a, b) = b` ;
- sinon, si `r` est le reste de la division euclidienne de `a` par `b`, alors `pgcd(a, b) = pgcd(b, r)`.

Version récursive compacte :

```python
def pgcd(a, b):
    return b if a % b == 0 else pgcd(b, a % b)
```

Cet algorithme se termine car le reste diminue strictement à chaque étape.

Les entrées sont `(a, b)` entiers > 0, la sortie est un entier = `pgcd(a, b)`.

In [ ]:
# Démonstration : PGCD d'Euclide

def pgcd(a, b):
    if b == 0:
        raise ValueError("b ne doit pas être nul")
    return b if a % b == 0 else pgcd(b, a % b)

print(pgcd(48, 18))  # 6
print(pgcd(100, 35)) # 5

## 6.2 Mesures d’efficacité — Complexité temporelle

Plusieurs critères possibles pour mesurer l’efficacité d’un algorithme :

- **temps d’exécution** (complexité temporelle) ;
- **mémoire utilisée** (variables, structures, pile d’appels...) ;
- **nombre d’accès en lecture/écriture** à une mémoire lente (clé USB, disque, réseau) ;
- **nombre d’appels réseau**, volume de données échangées.

Dans ce chapitre, on se concentre sur le **temps d’exécution** en fonction de la **taille de l’entrée** (souvent notée `n`).

On ne mesure pas en secondes, mais en **nombre d’opérations** (approché), puis on utilise la **notation O**.

## 6.3 Notation de Landau (notation O)

La notation de Landau sert à comparer des comportements **asymptotiques** (pour `n` grand).

Soient deux fonctions `f(n)` et `g(n)` positives pour `n` assez grand :

- `f(n) = O(g(n))` : il existe une constante `C > 0` et `n0` tels que pour tout `n >= n0`,
  `f(n) <= C * g(n)` ;
- `f(n) = Ω(g(n))` : `g(n) = O(f(n))` (minorée à un facteur près) ;
- `f(n) = Θ(g(n))` : `f(n) = O(g(n))` **et** `f(n) = Ω(g(n))` (croissance comparable) ;
- `f(n) = o(g(n))` : `f(n)/g(n) -> 0` quand `n -> ∞` (f négligeable devant g) ;
- `f(n) = ω(g(n))` : `f(n)/g(n) -> ∞` (f domine g).

Intuitivement :

- `O` ~ `<=` à un facteur constant près ;
- `Ω` ~ `>=` ;
- `Θ` ~ `=` (même ordre de grandeur) ;
- `o` ~ `<` ;
- `ω` ~ `>`.

Exemples d’ordres de grandeur (du plus lent au plus rapide) :

- `O(n!)` ;
- `O(2^n)` ;
- `O(n^3)` ;
- `O(n^2)` ;
- `O(n log n)` ;
- `O(n)` ;
- `O(log n)` ;
- `O(1)`.

### Exemples de complexité pour quelques algorithmes classiques

- accès à un élément d’un tableau : `Θ(1)` ;
- recherche dichotomique dans un tableau trié de taille `n` : `O(log n)` ;
- recherche séquentielle dans un tableau non trié : `O(n)` ;
- tri rapide (Quicksort) sur cas moyen : `O(n log n)` ;
- tri par sélection : `Θ(n^2)` ;
- tri par insertion : `O(n^2)` en général, mais `Ω(n)` sur une liste presque triée ;
- certains problèmes (comme le voyageur de commerce naïf) : `O(n!)`.

## 6.5 Algorithmes de tri

Nous allons voir plusieurs algorithmes de tri :

- **tri par sélection** (Selection Sort) ;
- **tri par insertion** (Insertion Sort) ;
- **tri rapide** (Quicksort).

Nous utiliserons des listes d’entiers générées aléatoirement pour les tests.

In [ ]:
# Utilitaires pour les tableaux de tests

from random import randrange

def make_random_array(n):
    """Renvoie une liste de n entiers aléatoires entre 0 et 10*n-1."""
    return [randrange(10 * n) for _ in range(n)]

def print_array(a, limit=100):
    """Affiche au plus 'limit' éléments de la liste a."""
    if len(a) <= limit:
        print(a)
    else:
        print(a[:limit], "...")

### 6.5.1 Tri par sélection (Selection Sort)

Idée :

1. Trouver le plus petit élément et le mettre en position 0 ;
2. Trouver le plus petit élément parmi les éléments restants et le mettre en position 1 ;
3. etc. jusqu’à l’avant‑dernier élément.

Complexité :

- nombre de comparaisons : `(n-1) + (n-2) + ... + 1 = n(n-1)/2` → `Θ(n^2)` ;
- indépendant de l’ordre initial (best, worst et average case ont même nombre de comparaisons).

In [ ]:
def selection_sort(a):
    """Trie la liste a en place (ordre croissant) par sélection."""
    n = len(a)
    for i in range(n - 1):
        p = i  # position du plus petit élément trouvé
        for j in range(i + 1, n):
            if a[j] < a[p]:
                p = j
        if p > i:
            a[i], a[p] = a[p], a[i]

# Exemple d'utilisation
a = make_random_array(10)
print("Avant tri (sélection) :", a)
selection_sort(a)
print("Après tri  (sélection) :", a)

### 6.5.5 Tri par insertion (Insertion Sort)

Idée :

On parcourt la liste de gauche à droite. Pour chaque élément `a[i]` :

- les éléments à gauche (`a[0..i-1]`) sont supposés déjà triés ;
- on **insère** `a[i]` à la bonne place dans cette sous‑liste triée ;
- pour cela, on décale vers la droite les éléments plus grands que `a[i]`.

Complexité :

- **worst case** (liste triée dans l’ordre inverse) : `O(n^2)` ;
- **best case** (liste déjà triée) : `O(n)` ;
- **average case** : `O(n^2)`.

Tri par insertion est intéressant pour des listes presque triées, ou comme complément dans un Quicksort optimisé.

In [ ]:
def insertion_sort(a):
    """Trie la liste a en place (ordre croissant) par insertion."""
    for i in range(1, len(a)):
        x = a[i]
        j = i
        while j > 0 and a[j - 1] > x:
            a[j] = a[j - 1]
            j -= 1
        if j < i:
            a[j] = x

# Exemple d'utilisation
b = make_random_array(10)
print("Avant tri (insertion) :", b)
insertion_sort(b)
print("Après tri  (insertion) :", b)

### 6.5.3 Tri rapide (Quicksort) — version simple

Idée :

1. Choisir un **pivot** (ici : le dernier élément de la sous‑liste) ;
2. **partitionner** la liste : tous les éléments < pivot à gauche, > pivot à droite ;
3. le pivot est à sa place définitive ;
4. appliquer récursivement le même procédé à la sous‑liste de gauche et à celle de droite.

Complexité :

- **cas moyen** : `O(n log n)` ;
- **meilleur cas** : `O(n log n)` ;
- **pire cas** (liste déjà triée et mauvais choix de pivot) : `O(n^2)`.

Nous implémentons la version avec pivot = dernier élément, comme dans le texte.

In [ ]:
def partition(a, g, d):
    """Partage la sous-liste a[g..d] autour d'un pivot (a[d]).
    Renvoie l'indice final du pivot."""
    p = a[d]
    i, j = g, d - 1
    while True:
        while i <= j and a[i] < p:
            i += 1
        while i <= j and a[j] > p:
            j -= 1
        if i <= j:
            a[i], a[j] = a[j], a[i]
            i += 1
            j -= 1
        else:
            # i et j se sont croisés : placer le pivot à sa position finale
            a[i], a[d] = a[d], a[i]
            return i

def quicksort(a, g=0, d=None):
    """Trie la liste a en place par Quicksort (pivot = dernier élément)."""
    if d is None:
        d = len(a) - 1
    if g >= d:
        return
    p = partition(a, g, d)
    quicksort(a, g, p - 1)
    quicksort(a, p + 1, d)

# Exemple d'utilisation
c = make_random_array(10)
print("Avant tri (quicksort) :", c)
quicksort(c)
print("Après tri  (quicksort) :", c)

# 6.X Exercices supplémentaires d'entraînement (avec grader)

Comme dans les chapitres précédents, cette section propose quelques exercices supplémentaires.

Un fichier **`grader_chapitre_6.py`** testera automatiquement les fonctions suivantes (sans montrer la solution) :

- `count_comparisons_selection` ;
- `count_comparisons_insertion` ;
- `is_sorted`.

Ces fonctions permettent d’expérimenter la **complexité** des tris sur des listes de différentes tailles.

## Exercice S1 — Compter les comparaisons du tri par sélection

**Temps conseillé : 15 à 20 minutes**

Écris une fonction :

```python
def count_comparisons_selection(a):
    ...
```

qui :

1. fait une **copie** de la liste `a` (pour ne pas modifier l’originale) ;
2. applique le tri par sélection sur cette copie ;
3. compte le **nombre total de comparaisons** `a[j] < a[p]` effectuées ;
4. renvoie ce nombre.

On attend que, pour `n = len(a)`, ce nombre soit toujours `n(n-1)/2`, indépendamment du contenu de `a`.

<details>
<summary><strong>Aide (dévoiler si besoin)</strong></summary>

- commence par `b = a.copy()` et travaille toujours sur `b` ;
- copie la structure de `selection_sort`, mais à chaque fois que tu effectues `if b[j] < b[p]:`, incrémente un compteur ;
- renvoie ce compteur à la fin.

</details>

## Exercice S2 — Compter les comparaisons du tri par insertion

**Temps conseillé : 15 à 20 minutes**

Écris une fonction :

```python
def count_comparisons_insertion(a):
    ...
```

qui :

1. fait une **copie** de la liste `a` ;
2. applique le tri par insertion sur cette copie ;
3. compte le **nombre total de comparaisons** `a[j-1] > x` effectuées dans la boucle `while` ;
4. renvoie ce nombre.

Tu pourras comparer :

- une liste déjà triée ;
- une liste triée à l’envers ;
- une liste aléatoire.

Cela illustre les notions de **best case**, **worst case**, **average case**.

<details>
<summary><strong>Aide (dévoiler si besoin)</strong></summary>

- crée une copie `b = a.copy()` et suis la structure de `insertion_sort` ;
- chaque fois que tu testes `while j > 0 and b[j - 1] > x:`, compte une **comparaison** pour `b[j-1] > x` ;
- même si la condition est fausse, tu as évalué `b[j-1] > x` une fois.

</details>

## Exercice S3 — Vérifier si une liste est triée

**Temps conseillé : 5 à 10 minutes**

Écris une fonction :

```python
def is_sorted(a):
    ...
```

qui renvoie `True` si la liste `a` est triée en **ordre croissant** (au sens `a[i] <= a[i+1]` pour tout `i`),
et `False` sinon.

Cette fonction est utile pour vérifier que les tris fonctionnent correctement.

<details>
<summary><strong>Aide (dévoiler si besoin)</strong></summary>

- parcours la liste avec `for i in range(len(a) - 1):` ;
- si tu trouves un `a[i] > a[i+1]`, renvoie immédiatement `False` ;
- si la boucle se termine sans problème, renvoie `True`.

</details>

## Comment utiliser le grader externe du chapitre 6

1. Assure‑toi d'avoir complété :
   - `count_comparisons_selection`,
   - `count_comparisons_insertion`,
   - `is_sorted`.

2. Sauvegarde ce notebook (`chapitre_6_interactif.ipynb`).

3. Dans un terminal, exécute le fichier **`grader_chapitre_6.py`** (placé à côté de ce notebook) :

```bash
python grader_chapitre_6.py
```

Le grader :

- importera ce notebook comme un module Python ;
- exécutera une série de tests cachés ;
- affichera pour chaque exercice un statut du type :
  - `S1: Réussi` ou `S1: Échoué`,
  - `S2: Réussi` ou `S2: Échoué`,
  - `S3: Réussi` ou `S3: Échoué`.

Tu sauras donc si ton implémentation est correcte, **sans voir la solution**.

In [ ]:
# Lancer le grader du chapitre 6 directement depuis ce notebook

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, importlib.util

BASE = "/content/drive/MyDrive/1ereB_info"
os.chdir(BASE)
print("Répertoire courant:", os.getcwd())

spec = importlib.util.spec_from_file_location(
    "grader_chapitre_6",
    os.path.join(BASE, "grader_chapitre_6.py"),
)
mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(mod)
mod.main()
